Train de PPO STUDENT with the env wrapper

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor

from env.overcooked_wrapper import OvercookedGym


def make_env():
    
    return Monitor(OvercookedGym("cramped_room"))


if __name__ == "__main__":
    env = DummyVecEnv([make_env])

    model = PPO(
        "MlpPolicy",
        env,
        learning_rate=3e-4,
        batch_size=64,
        n_steps=2048,
        verbose=1,
    )

    model.learn(total_timesteps=200_000)

    model.save("ppo_student_official")
    print("Training finished (official aligned env)")


Computing MediumLevelActionManager
Using cpu device
Computing MediumLevelActionManager
Computing MediumLevelActionManager
Computing MediumLevelActionManager
Computing MediumLevelActionManager
Computing MediumLevelActionManager
Computing MediumLevelActionManager
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 400      |
|    ep_rew_mean     | 0        |
| time/              |          |
|    fps             | 1556     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
Computing MediumLevelActionManager
Computing MediumLevelActionManager
Computing MediumLevelActionManager
Computing MediumLevelActionManager
Computing MediumLevelActionManager
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 400         |
|    ep_rew_mean          | 0           |
| time/                   |             |
|    fps     

Create an Overcooked Agent compatible (PPO Student Agent)

Evaluate student PPO & visualization

In [3]:
model = PPO.load("ppo_student_official")

In [4]:
obs = env.reset()
for i in range(20):
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    print(action, reward)


Computing MediumLevelActionManager
[1] [0.]
[1] [0.]
[1] [0.]
[1] [0.]
[1] [0.]
[1] [0.]
[3] [0.]
[0] [0.]
[1] [0.]
[3] [0.]
[0] [0.]
[5] [0.]
[3] [0.]
[4] [0.]
[4] [0.]
[4] [0.]
[4] [0.]
[4] [0.]
[4] [0.]
[4] [0.]


In [ ]:
from stable_baselines3 import PPO

from overcooked_ai_py.mdp.overcooked_mdp import OvercookedGridworld
from overcooked_ai_py.mdp.overcooked_env import OvercookedEnv
from overcooked_ai_py.agents.benchmarking import AgentEvaluator
from overcooked_ai_py.agents.agent import AgentPair
from overcooked_ai_py.visualization.state_visualizer import StateVisualizer

from student.ppo_student_agent import PPOStudentAgent



model = PPO.load("ppo_student_official")



mdp = OvercookedGridworld.from_layout_name("cramped_room", old_dynamics=True)
base_env = OvercookedEnv.from_mdp(mdp, horizon=400)
featurize_fn = base_env.featurize_state_mdp



agent0 = PPOStudentAgent(model, featurize_fn, agent_index=0)
agent1 = PPOStudentAgent(model, featurize_fn, agent_index=1)

pair = AgentPair(agent0, agent1, allow_duplicate_agents=True)



ae = AgentEvaluator.from_layout_name(
    mdp_params={"layout_name": "cramped_room", "old_dynamics": True},
    env_params={"horizon": 400},
)


trajs = ae.evaluate_agent_pair(pair, num_games=1)

print("Episode returns:", trajs["ep_returns"])


StateVisualizer().display_rendered_trajectory(trajs, ipython_display=True)


  0%|          | 0/1 [00:00<?, ?it/s]

Computing MediumLevelActionManager


Avg rew: 0.00 (std: 0.00, se: 0.00); avg len: 400.00; : 100%|██████████| 1/1 [00:00<00:00,  2.40it/s]


Episode returns: [0]


interactive(children=(IntSlider(value=0, description='timestep', max=399), Output()), _dom_classes=('widget-in…